In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 


In [11]:
df = pd.read_csv('Google_1.csv')

In [12]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,15.615220,15.678981,15.547723,15.610239,15.610239,78541293
1,2010-01-05,15.620949,15.637387,15.480475,15.541497,15.541497,120638494
2,2010-01-06,15.588072,15.588072,15.102393,15.149715,15.149715,159744526
3,2010-01-07,15.178109,15.193053,14.760922,14.797037,14.797037,257533695
4,2010-01-08,14.744733,15.024933,14.672753,14.994298,14.994298,189680313


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3589 entries, 0 to 3588
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3589 non-null   object 
 1   Open       3589 non-null   float64
 2   High       3589 non-null   float64
 3   Low        3589 non-null   float64
 4   Close      3589 non-null   float64
 5   Adj Close  3589 non-null   float64
 6   Volume     3589 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 196.4+ KB


In [15]:
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

<h1>

There is no missing values. 

</h1>